## Aprendizado de Máquina Federado
### Teste local do tensorflow_federated com simulação local de dispositivos de borda

In [1]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

2025-12-01 13:44:31.836245: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-01 13:44:31.891952: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-01 13:44:32.066037: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-01 13:44:32.066077: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-01 13:44:32.067097: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [3]:
# 1. Carregar o dataset Federado
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

# 2. Definir as funções de pré-processamento
NUM_CLIENTS = 10
BATCH_SIZE = 20
PREFETCH_BUFFER = 10

def preprocess(dataset):
    """Pré-processa o dataset para o modelo Keras."""

    # Mapear imagens para floats de 0-1 e redimensionar para vetores de 784
    def batch_format_fn(element):
        return (tf.reshape(element['pixels'], [-1, 784]),  # Imagens (28*28 = 784)
                tf.reshape(element['label'], [-1, 1]))    # Rótulos

    return dataset.batch(BATCH_SIZE).map(batch_format_fn).repeat(1).prefetch(PREFETCH_BUFFER)

# 3. Aplicar o pré-processamento nos dados de treinamento
# Vamos selecionar apenas um subconjunto de clientes para a simulação
sample_clients = emnist_train.client_ids[0:NUM_CLIENTS]

federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x)) 
                        for x in sample_clients]

In [4]:
def create_keras_model():
    """Cria um modelo Keras simples para classificação MNIST."""
    # O input é um vetor de 784 (28x28) pixels
    initializer = tf.keras.initializers.GlorotNormal(seed=1)
    
    return tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(784,)),
        tf.keras.layers.Dense(10, kernel_initializer=initializer), # 10 classes de saída (0-9)
        tf.keras.layers.Softmax(),
    ])

def model_fn():
    """Compila o modelo Keras e o empacota para TFF."""
    keras_model = create_keras_model()
    
    # O TFF requer um modelo Keras com um Optimizer para treinar localmente
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=federated_train_data[0].element_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [9]:
# CÓDIGO CORRETO PARA TFF v0.87.0 (API Simplificada)

# Construir o processo FedAvg
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    # Otimizador usado por CADA cliente para treinamento local
    client_optimizer_fn=tff.learning.optimizers.build_sgdm(learning_rate=0.01),
    # Otimizador usado pelo SERVIDOR para agregar os pesos
    server_optimizer_fn=tff.learning.optimizers.build_sgdm(learning_rate=1.0) 
)

# Inicializar o estado global do modelo
state = iterative_process.initialize()
print("Modelo Global Inicializado.")

Modelo Global Inicializado.


2025-12-01 13:58:18.412721: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-12-01 13:58:18.413183: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-12-01 13:58:18.438308: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-12-01 13:58:18.438619: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


In [10]:
NUM_ROUNDS = 10
print(f"Iniciando Simulação com {NUM_ROUNDS} Rodadas...")

for round_num in range(1, NUM_ROUNDS + 1):
    # Executar a próxima rodada de treinamento federado
    state, metrics = iterative_process.next(state, federated_train_data)
    
    # Exibir as métricas de desempenho agregadas no servidor
    train_metrics = metrics['client_work']['train']
    
    print(f"\n--- Rodada {round_num:2d} ---")
    print(f"  Perda (Loss) agregada: {train_metrics['loss']:.4f}")
    print(f"  Acurácia agregada: {train_metrics['sparse_categorical_accuracy']:.4f}")

Iniciando Simulação com 10 Rodadas...


2025-12-01 13:59:25.197994: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-12-01 13:59:25.198186: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-12-01 13:59:25.248861: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-12-01 13:59:25.249062: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-12-01 13:59:25.254477: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-12-01 13:59:25.254709: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-12-01 13:59:25.262579: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2025-12-01 13:59:25.262815: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session



--- Rodada  1 ---
  Perda (Loss) agregada: 3.1233
  Acurácia agregada: 0.0741

--- Rodada  2 ---
  Perda (Loss) agregada: 2.8905
  Acurácia agregada: 0.0628

--- Rodada  3 ---
  Perda (Loss) agregada: 2.8346
  Acurácia agregada: 0.0658

--- Rodada  4 ---
  Perda (Loss) agregada: 2.8314
  Acurácia agregada: 0.0689

--- Rodada  5 ---
  Perda (Loss) agregada: 2.8129
  Acurácia agregada: 0.0720

--- Rodada  6 ---
  Perda (Loss) agregada: 2.7988
  Acurácia agregada: 0.0730

--- Rodada  7 ---
  Perda (Loss) agregada: 2.7835
  Acurácia agregada: 0.0730

--- Rodada  8 ---
  Perda (Loss) agregada: 2.7687
  Acurácia agregada: 0.0741

--- Rodada  9 ---
  Perda (Loss) agregada: 2.7538
  Acurácia agregada: 0.0751

--- Rodada 10 ---
  Perda (Loss) agregada: 2.7391
  Acurácia agregada: 0.0782
